In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_raw = pd.read_csv(f'/content/drive/MyDrive/SAP_CASE_GRIND/SAP Datasets.csv', encoding='latin-1')

<ipython-input-4-86411872a35d>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(f'/content/drive/MyDrive/SAP_CASE_GRIND/SAP Datasets.csv', encoding='latin-1')


In [6]:
!pip install pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.6 MB/s eta 0:00:00


In [8]:
import pycountry

file_path = "/content/drive/MyDrive/SAP_CASE_GRIND/SAP Datasets.csv"
df = pd.read_csv(file_path, encoding='latin-1')

print("Initial shape of the dataset:", df.shape)

country_column = "Country Name"

# Valid countries
valid_countries = [country.name for country in pycountry.countries]

#Clean the data
df_clean = df[df[country_column].isin(valid_countries)]
print("After filtering using pycountry (valid countries only):", df_clean.shape)


# region_keywords = ['income', 'IDA', 'IBRD', 'Asia', 'Africa', 'Americas', 'Europe', 'Oceania']

# def contains_region_keyword(name):
#     """Return True if the name contains any region-related keyword."""
#     for keyword in region_keywords:
#         if keyword.lower() in name.lower():
#             return True
#     return False


# df_clean = df_clean[~df_clean[country_column].apply(contains_region_keyword)]
# print("After filtering out region keywords:", df_clean.shape)


year_columns = [str(year) for year in range(2000, 2024)]
print("Year columns identified:", year_columns)

# filtering if 70% of data is missing
missing_threshold = 17

df_clean = df_clean[df_clean[year_columns].isnull().sum(axis=1) < missing_threshold]
print("After filtering rows with excessive missing year data:", df_clean.shape)

df_clean[year_columns] = df_clean[year_columns].apply(lambda x: x.fillna(x.mean()), axis=0)

#save
output_file = "/content/drive/MyDrive/SAP_CASE_GRIND/cleaned_data.csv"
df_clean.to_csv(output_file, index=False)
print(f"Cleaned data saved to '{output_file}'")


<ipython-input-8-20366d8a8bee>:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


Initial shape of the dataset: (23141, 32)
After filtering using pycountry (valid countries only): (16182, 32)
Year columns identified: ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
After filtering rows with excessive missing year data: (8129, 32)
Cleaned data saved to '/content/drive/MyDrive/SAP_CASE_GRIND/cleaned_data.csv'


In [9]:
pd.read_csv("/content/drive/MyDrive/SAP_CASE_GRIND/cleaned_data.csv")

,Country Name,Country Code,Indicator Name,Topic,short description,long description,Indicator Code,Unit of measure,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,Control of Corruption: Estimate,Public Sector: Policy & institutions,NaN,NaN,CC.EST,0,1.987590e+10,1.641299e+10,...,1.014232,1.248461,1.232735,1.239377,1.207429,1.174703,1.199392,0.801611,0.756785,-8.552220e+10
1,Afghanistan,AFG,Control of Corruption: Estimate,Public Sector: Policy & institutions,NaN,NaN,CC.EST,0,-1.271724e+00,1.641299e+10,...,-1.364743,-1.354240,-1.540353,-1.531910,-1.502881,-1.419499,-1.493700,-1.152327,-1.183776,-8.552220e+10
2,Angola,AGO,Control of Corruption: Estimate,Public Sector: Policy & institutions,NaN,NaN,CC.EST,0,-1.197514e+00,1.641299e+10,...,-1.457798,-1.427323,-1.483337,-1.444333,-1.199251,-1.058418,-0.938673,-0.651610,-0.601941,-8.552220e+10
3,Albania,ALB,Control of Corruption: Estimate,Public Sector: Policy & institutions,NaN,NaN,CC.EST,0,-8.555636e-01,1.641299e+10,...,-0.586141,-0.544385,-0.471469,-0.480891,-0.545840,-0.563900,-0.572924,-0.576536,-0.407876,-8.552220e+10
4,Andorra,AND,Control of Corruption: Estimate,Public Sector: Policy & institutions,NaN,NaN,CC.EST,0,1.313404e+00,1.641299e+10,...,1.220709,1.151811,1.159556,1.171758,1.179166,1.176592,1.266005,1.279872,1.270204,-8.552220e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8124,Vanuatu,VUT,Urban population (% of total population),Environment: Density & urbanization,NaN,Urban population refers to people living in ur...,SP.URB.TOTL.IN.ZS,0,2.167300e+01,2.195300e+01,...,24.861000,24.961000,25.062000,25.163000,25.274000,25.394000,25.525000,25.665000,25.816000,2.597600e+01
8125,Samoa,WSM,Urban population (% of total population),Environment: Density & urbanization,NaN,Urban population refers to people living in ur...,SP.URB.TOTL.IN.ZS,0,2.197700e+01,2.206800e+01,...,19.149000,18.914000,18.681000,18.452000,18.243000,18.056000,17.889000,17.742000,17.616000,1.750800e+01
8126,South Africa,ZAF,Urban population (% of total population),Environment: Density & urbanization,NaN,Urban population refers to people living in ur...,SP.URB.TOTL.IN.ZS,0,5.689100e+01,5.736800e+01,...,64.312000,64.828000,65.341000,65.850000,66.355000,66.856000,67.354000,67.847000,68.335000,6.881900e+01
8127,Zambia,ZMB,Urban population (% of total population),Environment: Density & urbanization,NaN,Urban population refers to people living in ur...,SP.URB.TOTL.IN.ZS,0,3.480200e+01,3.500200e+01,...,41.382000,41.907000,42.438000,42.976000,43.521000,44.072000,44.629000,45.192000,45.761000,4.633500e+01
